# Prerequisites
* Store Following api keys in `secrets` of google colab

  1. Huggingface access token with name `HF_TOKEN`.
  2.  `Pinecone API` with name `PINECONE_API_KEY`.

In [35]:
!pip install -qqq -U langchain_chroma
!pip install chromadb
!pip install -q -U FlagEmbedding
!pip install -qqq -U langchain-huggingface
!pip install -qqq -U langchain_core
!pip install -qqq -U langchain-community
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.1 MB/s eta 0:00:00


# Chroma Database

Chroma DB is a vector database system that allows you to store, retrieve, and manage embeddings. It can be used in Python or JavaScript with the chromadb library for local use, or connected to a remote server running Chroma. Users can configure Chroma to persist data on disk and create collections of embeddings using unique names.

In [1]:
import chromadb
from chromadb.config import Settings
from FlagEmbedding import FlagModel
import os
from langchain_core.documents import Document

ModuleNotFoundError: No module named 'FlagEmbedding'

In [ ]:
model = FlagModel("BAAI/bge-large-en",
                  query_instruction_for_retrieval="Represent this sentence for searching relevant passages",
                  use_fp16=False)

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [ ]:
sentences_1 = "Gujarat, located on the western coast of India, is rich in ancient history. Its settlements date back to the Indus Valley Civilization, with cities like Dholavira showing advanced urban planning. Over 4,000 years ago, it became a center of maritime trade. Gujarat played a crucial role in the spread of culture across the Arabian Sea, with links to Mesopotamia, the Persian Gulf, and even Africa."
sentences_2 = "The history of human settlement in Rajasthan goes back more than 100,000 years, with the Indus Valley Civilization marking one of the earliest urban settlements. Sites like Kalibangan show evidence of fire altars, and Rajasthan's strategic location made it a hub for trade. The region's culture and history have been shaped by its rich archaeological past, which continues to be explored."
sentences_3 = "Uttar Pradesh, a state in northern India, has a diverse economy dominated by agriculture, services, and manufacturing. The state is a significant contributor to India’s agriculture sector, producing crops like wheat, rice, and sugarcane. In recent years, the economy has shifted toward industrial growth, including sectors like textiles, electronics, and infrastructure development. The state is also a key player in India’s political landscape."

In [ ]:
database_folder = 'database'
collection_name = 'genAI'

client = chromadb.PersistentClient(path=database_folder,settings=Settings(allow_reset=True))
client.reset()
collection = client.create_collection(name=collection_name)

In [ ]:
collection.add(
    documents=[sentences_1, sentences_2, sentences_3],
    metadatas=[{"source": "Wikipedia"}, {"source": "google"}, {"source": "google"}],
    ids=["doc1", "doc2", "doc3"]
)
print("New collection created and documents added.")

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 104MiB/s]


New collection created and documents added.


In [ ]:
results = collection.query(
    query_texts=[
        "Something about Gujarat"
    ],
    n_results=2
)

print(results)

{'ids': [['doc1', 'doc3']], 'embeddings': None, 'documents': [['Gujarat, located on the western coast of India, is rich in ancient history. Its settlements date back to the Indus Valley Civilization, with cities like Dholavira showing advanced urban planning. Over 4,000 years ago, it became a center of maritime trade. Gujarat played a crucial role in the spread of culture across the Arabian Sea, with links to Mesopotamia, the Persian Gulf, and even Africa.', 'Uttar Pradesh, a state in northern India, has a diverse economy dominated by agriculture, services, and manufacturing. The state is a significant contributor to India’s agriculture sector, producing crops like wheat, rice, and sugarcane. In recent years, the economy has shifted toward industrial growth, including sectors like textiles, electronics, and infrastructure development. The state is also a key player in India’s political landscape.']], 'uris': None, 'data': None, 'metadatas': [[{'source': 'Wikipedia'}, {'source': 'google

## Using Langchain chroma

### Adding texts




In [ ]:
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
db_name = "bge_db"

if os.path.exists(db_name):
  Chroma(persist_directory = db_name,embedding_function = model).delete_collection()

texts = [sentences_1, sentences_2, sentences_3]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

chunks_per_text = [text_splitter.split_text(text) for text in texts]
flat_chunks = [chunk for text_chunks in chunks_per_text for chunk in text_chunks]

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en",encode_kwargs={'normalize_embeddings':True})
text_vectorstore = Chroma.from_texts(
    texts=flat_chunks,
    embedding=embeddings,
    persist_directory=db_name
)

print(f"Chroma Vectorstore created with {text_vectorstore._collection.count()} chunks")

Chroma Vectorstore created with 3 chunks


In [ ]:
retriever_text = text_vectorstore.as_retriever(search_type="similarity",search_kwargs={"k": 2})

In [ ]:
retriever_text.get_relevant_documents("Something about Gujarat")

[Document(metadata={}, page_content='Gujarat, located on the western coast of India, is rich in ancient history. Its settlements date back to the Indus Valley Civilization, with cities like Dholavira showing advanced urban planning. Over 4,000 years ago, it became a center of maritime trade. Gujarat played a crucial role in the spread of culture across the Arabian Sea, with links to Mesopotamia, the Persian Gulf, and even Africa.'),
 Document(metadata={}, page_content="The history of human settlement in Rajasthan goes back more than 100,000 years, with the Indus Valley Civilization marking one of the earliest urban settlements. Sites like Kalibangan show evidence of fire altars, and Rajasthan's strategic location made it a hub for trade. The region's culture and history have been shaped by its rich archaeological past, which continues to be explored.")]

### Adding documents

In [ ]:
document_1 = Document(
    page_content=sentences_1,
    metadata={"source": "tweet"},
)
document_2 = Document(
    page_content=sentences_2,
    metadata={"source": "tweet"},
)
document_3 = Document(
    page_content=sentences_3,
    metadata={"source": "google"},
)

In [ ]:
db_name = "bge_db_documents"

if os.path.exists(db_name):
  Chroma(persist_directory = db_name,embedding_function = model).delete_collection()

docs = [document_1, document_2,document_3]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

chunks=text_splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en",encode_kwargs={'normalize_embeddings':True})
vectorstore_documents = Chroma.from_documents(
    documents = chunks,
    embedding=embeddings,
    persist_directory=db_name
)

print(f"Chroma Vectorstore created with {vectorstore_documents._collection.count()} chunks")

Chroma Vectorstore created with 3 chunks


#### Using as retriever to search documents (Useful in Langchain)

In [ ]:
retriever_doc = vectorstore_documents.as_retriever(search_type="similarity",search_kwargs={"k": 2})

In [ ]:
retriever_doc.get_relevant_documents("Something about Gujarat")

[Document(metadata={'source': 'tweet'}, page_content='Gujarat, located on the western coast of India, is rich in ancient history. Its settlements date back to the Indus Valley Civilization, with cities like Dholavira showing advanced urban planning. Over 4,000 years ago, it became a center of maritime trade. Gujarat played a crucial role in the spread of culture across the Arabian Sea, with links to Mesopotamia, the Persian Gulf, and even Africa.'),
 Document(metadata={'source': 'tweet'}, page_content="The history of human settlement in Rajasthan goes back more than 100,000 years, with the Indus Valley Civilization marking one of the earliest urban settlements. Sites like Kalibangan show evidence of fire altars, and Rajasthan's strategic location made it a hub for trade. The region's culture and history have been shaped by its rich archaeological past, which continues to be explored.")]

#### Query directly on vectorstore

In [ ]:
vectorstore_documents.similarity_search(
    "Uttar Pradesh",
    k=2,
)

[Document(metadata={'source': 'google'}, page_content='Uttar Pradesh, a state in northern India, has a diverse economy dominated by agriculture, services, and manufacturing. The state is a significant contributor to India’s agriculture sector, producing crops like wheat, rice, and sugarcane. In recent years, the economy has shifted toward industrial growth, including sectors like textiles, electronics, and infrastructure development. The state is also a key player in India’s political landscape.'),
 Document(metadata={'source': 'tweet'}, page_content="The history of human settlement in Rajasthan goes back more than 100,000 years, with the Indus Valley Civilization marking one of the earliest urban settlements. Sites like Kalibangan show evidence of fire altars, and Rajasthan's strategic location made it a hub for trade. The region's culture and history have been shaped by its rich archaeological past, which continues to be explored.")]

### Similarity search with score

In [ ]:
results = vectorstore_documents.similarity_search_with_score(
    "Uttar Pradesh", k=1
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.303897] Uttar Pradesh, a state in northern India, has a diverse economy dominated by agriculture, services, and manufacturing. The state is a significant contributor to India’s agriculture sector, producing crops like wheat, rice, and sugarcane. In recent years, the economy has shifted toward industrial growth, including sectors like textiles, electronics, and infrastructure development. The state is also a key player in India’s political landscape. [{'source': 'google'}]


# Facebook AI Similarity Search (Faiss) Database

 FAISS is an open-source library built for similarity search and clustering of dense vectors. Faiss can be used to build an index and perform searches with remarkable speed and memory efficiency. Additionally, it enhances search performance through its GPU implementations for various indexing methods.

In [ ]:
from langchain.vectorstores import FAISS

### Adding texts

In [ ]:
faiss_text_vectorstore = FAISS.from_texts(flat_chunks, embeddings) # Creates an in memory docstore

In [ ]:
faiss_text_retriever =faiss_text_vectorstore.as_retriever(search_type="similarity",search_kwargs={"k": 2})

In [ ]:
faiss_text_retriever.get_relevant_documents("Something about Rajasthan")

[Document(id='9b62fa0a-46b8-4a95-b3aa-cbd68bfcae8d', metadata={}, page_content="The history of human settlement in Rajasthan goes back more than 100,000 years, with the Indus Valley Civilization marking one of the earliest urban settlements. Sites like Kalibangan show evidence of fire altars, and Rajasthan's strategic location made it a hub for trade. The region's culture and history have been shaped by its rich archaeological past, which continues to be explored."),
 Document(id='a6868d91-0020-473a-bb61-17cb9e9a828a', metadata={}, page_content='Gujarat, located on the western coast of India, is rich in ancient history. Its settlements date back to the Indus Valley Civilization, with cities like Dholavira showing advanced urban planning. Over 4,000 years ago, it became a center of maritime trade. Gujarat played a crucial role in the spread of culture across the Arabian Sea, with links to Mesopotamia, the Persian Gulf, and even Africa.')]

### Adding documents

In [ ]:
faiss_vectorstore_doc = FAISS.from_documents(documents= chunks, embedding=embeddings)

#### Using as retriever to search documents (Useful in Langchain)

In [ ]:
faiss_retriever_doc =faiss_vectorstore_doc.as_retriever(search_type="similarity",search_kwargs={"k": 2})

In [ ]:
faiss_retriever_doc.get_relevant_documents("Something about Uttar Pradesh")

[Document(id='223511ad-1444-4848-90be-749d090df190', metadata={'source': 'google'}, page_content='Uttar Pradesh, a state in northern India, has a diverse economy dominated by agriculture, services, and manufacturing. The state is a significant contributor to India’s agriculture sector, producing crops like wheat, rice, and sugarcane. In recent years, the economy has shifted toward industrial growth, including sectors like textiles, electronics, and infrastructure development. The state is also a key player in India’s political landscape.'),
 Document(id='39d552ce-f8f5-4f3c-ab70-da9ee469c72f', metadata={'source': 'tweet'}, page_content="The history of human settlement in Rajasthan goes back more than 100,000 years, with the Indus Valley Civilization marking one of the earliest urban settlements. Sites like Kalibangan show evidence of fire altars, and Rajasthan's strategic location made it a hub for trade. The region's culture and history have been shaped by its rich archaeological past,

#### Query directly on vectorstore

In [ ]:
faiss_vectorstore_doc.similarity_search(
    "Uttar Pradesh",
    k=2,
    # filter={"source": "tweet"}, # optional params
)

[Document(id='223511ad-1444-4848-90be-749d090df190', metadata={'source': 'google'}, page_content='Uttar Pradesh, a state in northern India, has a diverse economy dominated by agriculture, services, and manufacturing. The state is a significant contributor to India’s agriculture sector, producing crops like wheat, rice, and sugarcane. In recent years, the economy has shifted toward industrial growth, including sectors like textiles, electronics, and infrastructure development. The state is also a key player in India’s political landscape.'),
 Document(id='39d552ce-f8f5-4f3c-ab70-da9ee469c72f', metadata={'source': 'tweet'}, page_content="The history of human settlement in Rajasthan goes back more than 100,000 years, with the Indus Valley Civilization marking one of the earliest urban settlements. Sites like Kalibangan show evidence of fire altars, and Rajasthan's strategic location made it a hub for trade. The region's culture and history have been shaped by its rich archaeological past,

### Similarity search with score

In [ ]:
results = faiss_vectorstore_doc.similarity_search_with_score(
    "Uttar Pradesh", k=1
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.303897] Uttar Pradesh, a state in northern India, has a diverse economy dominated by agriculture, services, and manufacturing. The state is a significant contributor to India’s agriculture sector, producing crops like wheat, rice, and sugarcane. In recent years, the economy has shifted toward industrial growth, including sectors like textiles, electronics, and infrastructure development. The state is also a key player in India’s political landscape. [{'source': 'google'}]


### Saving and loading

* We can also save and load a FAISS index. This is useful so we don't have to recreate it everytime you use it.

In [ ]:
faiss_vectorstore_doc.save_local("faiss_index")

new_vector_store = FAISS.load_local(
    "faiss_index", embeddings, allow_dangerous_deserialization=True
)

docs = new_vector_store.similarity_search(" Uttar Pradesh")

In [ ]:
docs

[Document(id='223511ad-1444-4848-90be-749d090df190', metadata={'source': 'google'}, page_content='Uttar Pradesh, a state in northern India, has a diverse economy dominated by agriculture, services, and manufacturing. The state is a significant contributor to India’s agriculture sector, producing crops like wheat, rice, and sugarcane. In recent years, the economy has shifted toward industrial growth, including sectors like textiles, electronics, and infrastructure development. The state is also a key player in India’s political landscape.'),
 Document(id='39d552ce-f8f5-4f3c-ab70-da9ee469c72f', metadata={'source': 'tweet'}, page_content="The history of human settlement in Rajasthan goes back more than 100,000 years, with the Indus Valley Civilization marking one of the earliest urban settlements. Sites like Kalibangan show evidence of fire altars, and Rajasthan's strategic location made it a hub for trade. The region's culture and history have been shaped by its rich archaeological past,

# Pinecone

To get started, simply head over to [pinecone.io](https://www.pinecone.io/) and sign up for a free account get an `API key`.

## Key Concepts in Pinecone

* **Pinecone index:**  represents the top-level organizational structure for vector data. It is responsible for receiving and storing vectors, handling queries on the stored vectors, and performing various vector operations within its data. Each index operates on one or more pods to ensure efficient functionality.

* **Pods:** are pre-configured hardware units that host Pinecone services. Each index in Pinecone operates on one or multiple pods, and having additional pods generally results in increased storage capacity, reduced latency, and improved throughput. Furthermore, users have the flexibility to create pods of varying sizes to meet specific requirements.

* **Distance Metrics:** Distance Metrics are mathematical techniques used to assess the similarity between two vectors within a vector space. In vector databases, these measures play a crucial role in comparing the stored vectors with a given query vector to identify the most similar ones.

  *we can choose from different metrics when creating a vector index:*

  1. **Cosine similarity:** This measure evaluates the cosine of the angle between two vectors in the vector space. Its scale ranges from -1 to 1, where 1 signifies identical vectors, 0 represents orthogonal vectors, and -1 indicates diametrically opposed vectors.
  2. **Euclidean distance:** This measure calculates the straight-line distance between two vectors in the vector space. It ranges from 0 to infinity, where 0 denotes identical vectors, and larger values indicate increasingly dissimilar vectors.
  3. **Dot product:** This measure computes the product of the magnitudes of two vectors and the cosine of the angle between them. Its scale spans from -∞ to ∞, where positive values indicate vectors pointing in the same direction, 0 represents orthogonal vectors, and negative values signify vectors pointing in opposite directions.


## **Serverless Indexes:**
 With serverless indexes, we don’t configure or manage any compute or storage resources. Instead, based on a breakthrough architecture, serverless indexes scale automatically based on usage, and we pay only for the amount of data stored and operations performed, with no minimums. This means that there’s no extra cost for having additional indexes.
***Ex:*** `aws, gcp, azure`

---
**Note:**

On the free Starter plan, you can create serverless indexes in the `us-east-1` region of `AWS` only. To create indexes in other regions, upgrade your plan.

---


## **Pod-based indexes:**
 With pod-based indexes, we choose one or more pre-configured units of hardware (pods). Depending on the pod type, pod size, and number of pods used, we get different amounts of storage and higher or lower latency and throughput. Be sure to choose an appropriate pod type and size for our dataset and workload.

### **Pod Types:**
  1. **s1 pods:**  it has enough capacity for around 5M vectors of 768 dimensions.
  2. **p2 pods:** it has enough capacity for around 1M vectors of 768 dimensions

### **Pod size and performance:**
Each pod type supports four pod sizes: `x1`, `x2`, `x4`, and `x8`. The index storage and compute capacity doubles for each size step. The default pod size is `x1`. We can increase the size of a pod after index creation.

### **[Pod environments](https://docs.pinecone.io/guides/indexes/understanding-indexes#pod-environments)**



REF:
1. https://github.com/pinecone-io/pinecone-python-client
2. https://docs.pinecone.io/guides/get-started/overview

In [26]:
!pip install "pinecone[grpc]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.2
    Uninstalling protobuf-5.29.2:
      Successfully uninstalled protobuf-5.29.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.29.0 requires protobuf<6.0,>=5.0, but you have protobuf 4.25.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: pinecone-plugin-inference
    Found existing installation: pinecone-plugin-inference 3.1.0
    Uninstalling pinecone-plugin-inference-3.1.0:
      Successfully uninstalled pinecone-plugin-inference-3.1.0
  Attempti

In [11]:
from pinecone import Pinecone,ServerlessSpec
from google.colab import userdata
import time

In [12]:
# Initialize client
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
pc = Pinecone(api_key=PINECONE_API_KEY)

In [13]:
# list existing indexes
pc.list_indexes()

[]

## Creating serverless index

In [29]:
index_name = "genai-serverless"  #Allowed characters are lowercase letters, numbers, and hyphens and the name may not begin or end with hyphens. Maximum length is 45 characters.
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

### Upsert data

 upsert operation is used to **`write`** records into a `namespace` in an index. [**Namespaces**](https://docs.pinecone.io/guides/get-started/key-features#namespaces) let you partition records within an index and are essential for [implementing multitenancy](https://docs.pinecone.io/guides/get-started/implement-multitenancy) when you need to isolate the data of each customer/user.

 * If a record ID already exists, upsert overwrites the entire record. To update only part of a record, use the [**`update`**](https://docs.pinecone.io/guides/data/update-data) operation instead."
 * When upserting larger amounts of data, it is recommended to `upsert records in large batches`. A batch of upserts should be as large as possible (up to 1000 records) without exceeding the maximum request size of 2MB.

  To understand the number of records you can fit into one batch based on the vector dimensions and metadata size, see the following table:
```
  | Dimension | Metadata (bytes) | Max batch size |
  |-----------|------------------|----------------|
  | 386       | 0                | 1000           |
  | 768       | 500              | 559            |
  | 1536      | 2000             | 245            |
```

* To use upsert first get the **`index host`** using pinecone api.
To get an index host from the Pinecone API, use the **`describe_index`** operation, which returns the index host as the host value

In [32]:
host = pc.describe_index(index_name)['host']
index = pc.Index(index_name,host=host)

#### set up embeddings

In [36]:
from FlagEmbedding import FlagModel
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [37]:
model = FlagModel("BAAI/bge-large-en",
                  query_instruction_for_retrieval="Represent this sentence for searching relevant passages",
                  use_fp16=False)

In [39]:
sentences_1 = "Gujarat, located on the western coast of India, is rich in ancient history. Its settlements date back to the Indus Valley Civilization, with cities like Dholavira showing advanced urban planning. Over 4,000 years ago, it became a center of maritime trade. Gujarat played a crucial role in the spread of culture across the Arabian Sea, with links to Mesopotamia, the Persian Gulf, and even Africa."
sentences_2 = "The history of human settlement in Rajasthan goes back more than 100,000 years, with the Indus Valley Civilization marking one of the earliest urban settlements. Sites like Kalibangan show evidence of fire altars, and Rajasthan's strategic location made it a hub for trade. The region's culture and history have been shaped by its rich archaeological past, which continues to be explored."
sentences_3 = "Uttar Pradesh, a state in northern India, has a diverse economy dominated by agriculture, services, and manufacturing. The state is a significant contributor to India’s agriculture sector, producing crops like wheat, rice, and sugarcane. In recent years, the economy has shifted toward industrial growth, including sectors like textiles, electronics, and infrastructure development. The state is also a key player in India’s political landscape."

In [40]:
texts = [sentences_1, sentences_2, sentences_3]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

chunks_per_text = [text_splitter.split_text(text) for text in texts]
flat_chunks = [chunk for text_chunks in chunks_per_text for chunk in text_chunks]

In [41]:
len(flat_chunks)

3

In [42]:
embedding_vector_1 = model.encode(flat_chunks[0])
embedding_vector_2 = model.encode(flat_chunks[1])
embedding_vector_3 = model.encode(flat_chunks[2])

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


#### upsert data individually

In [46]:
# upsert data in namespace
namespace="genai_upsert_individual"
index.upsert(
  vectors=[
    {
      "id": "A",
      "values": embedding_vector_1,
      "metadata": {"type": "gujarat"}
    },
    {
      "id": "B",
      "values": embedding_vector_2,
      "metadata": {"type": "rajasthan"}
    },
    {
      "id": "C",
      "values": embedding_vector_3,
      "metadata": {"type": "uttarpradesh"}
    },

  ],
  namespace=namespace
)

{'upserted_count': 3}

#### Other Ways to Upsert:

1. [in batches synchronously](https://docs.pinecone.io/guides/data/upsert-data#upsert-records-in-batches)
2. [Send upserts in parallel](https://docs.pinecone.io/guides/data/upsert-data#send-upserts-in-parallel)
3. [Upsert a dataset as a dataframe](https://docs.pinecone.io/guides/data/upsert-data#upsert-a-dataset-as-a-dataframe)


### Query data

#### By record ID

In [33]:
index.query(
    namespace='genai_upsert_individual',
   id="B",
    top_k=3,
    include_values=True
)

{'matches': [{'id': 'B',
              'score': 0.999809623,
              'values': [0.0122622279,
                         0.0290986989,
                         -0.0154122803,
                         -0.00380640081,
                         -0.0134122893,
                         0.0141480751,
                         -0.0304043572,
                         0.0182409361,
                         0.0614665821,
                         0.00239238702,
                         0.0574615858,
                         -0.033170335,
                         -0.000810960773,
                         -0.0363697261,
                         0.00773921935,
                         -0.00157843938,
                         -0.0398735665,
                         -0.00537199713,
                         -0.0325756259,
                         0.0283954199,
                         8.81734e-05,
                         0.0335909091,
                         -0.0700588897,
                         

#### by query vector

In [38]:
query_vector = model.encode("Something about Gujarat")
query_vector_embedding_list = query_vector.tolist()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [39]:
index.query(
    namespace='genai_upsert_individual',
    vector=query_vector_embedding_list,
    top_k=3,
    include_metadata=True
)
print(len(flat_chunks))

{'matches': [{'id': 'A',
              'metadata': {'type': 'gujarat'},
              'score': 0.851787627,
              'values': []},
             {'id': 'B',
              'metadata': {'type': 'rajasthan'},
              'score': 0.778967738,
              'values': []},
             {'id': 'C',
              'metadata': {'type': 'uttarpradesh'},
              'score': 0.75601095,
              'values': []}],
 'namespace': 'genai_upsert_individual',
 'usage': {'read_units': 6}}

#### Query with metadata filters

In [40]:
index.query(
    namespace='genai_upsert_individual',
    vector=query_vector_embedding_list,
    filter={
        "type": "gujarat"
    },
    top_k=1,
    include_metadata=True # Include metadata in the response.
)

{'matches': [{'id': 'A',
              'metadata': {'type': 'gujarat'},
              'score': 0.851787627,
              'values': []}],
 'namespace': 'genai_upsert_individual',
 'usage': {'read_units': 6}}

### Fetch Data


In [47]:
index.fetch(ids=["A"], namespace='genai_upsert_individual')

{'namespace': 'genai_upsert_individual',
 'usage': {'read_units': 1},
 'vectors': {'A': {'id': 'A',
                   'metadata': {'available': True, 'type': 'gujarat'},
                   'values': [0.0303250607,
                              0.0299873818,
                              -0.0168242585,
                              -0.0365483463,
                              -0.0279876459,
                              0.00231287978,
                              -0.0102339918,
                              0.0170698818,
                              0.0277173892,
                              0.00997042563,
                              0.0142245973,
                              -0.0345505923,
                              -0.00337819033,
                              -0.0410457179,
                              -0.00682543078,
                              -0.00361718726,
                              -0.0324576162,
                              -0.0262875222,
                     

### Update data

use `index.update()`

In [46]:
index.update(
	id="A",
	set_metadata={"type": "Gujarat"},  # capitalize the G of gujarat
	namespace="genai_upsert_individual"
)

{}

### Delete Data

In [48]:
index.delete(ids=["A"], namespace='genai_upsert_individual')

{}

In [49]:
index.fetch(ids=["A"], namespace='genai_upsert_individual')

{'namespace': 'genai_upsert_individual',
 'usage': {'read_units': 1},
 'vectors': {}}

after deletion we can't find the vector with id as **`A`**

## Langchain Pinecone

In [6]:
!pip install langchain_pinecone
!pip install langchain_huggingface

In [2]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec
from google.colab import userdata
import time

In [3]:
# Initialize client
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
pc = Pinecone(api_key=PINECONE_API_KEY)

## Create Index

In [5]:
index_name = "genai-serverless"  #Allowed characters are lowercase letters, numbers, and hyphens and the name may not begin or end with hyphens. Maximum length is 45 characters.
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

host = pc.describe_index(index_name)['host']

## access index
index = pc.Index(index_name,host=host)

### setup data

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
import os
from langchain_core.documents import Document

In [8]:
sentences_1 = "Gujarat, located on the western coast of India, is rich in ancient history. Its settlements date back to the Indus Valley Civilization, with cities like Dholavira showing advanced urban planning. Over 4,000 years ago, it became a center of maritime trade. Gujarat played a crucial role in the spread of culture across the Arabian Sea, with links to Mesopotamia, the Persian Gulf, and even Africa."
sentences_2 = "The history of human settlement in Rajasthan goes back more than 100,000 years, with the Indus Valley Civilization marking one of the earliest urban settlements. Sites like Kalibangan show evidence of fire altars, and Rajasthan's strategic location made it a hub for trade. The region's culture and history have been shaped by its rich archaeological past, which continues to be explored."
sentences_3 = "Uttar Pradesh, a state in northern India, has a diverse economy dominated by agriculture, services, and manufacturing. The state is a significant contributor to India’s agriculture sector, producing crops like wheat, rice, and sugarcane. In recent years, the economy has shifted toward industrial growth, including sectors like textiles, electronics, and infrastructure development. The state is also a key player in India’s political landscape."

#### as text

In [15]:
texts = [sentences_1, sentences_2, sentences_3]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

chunks_per_text = [text_splitter.split_text(text) for text in texts]
flat_chunks = [chunk for text_chunks in chunks_per_text for chunk in text_chunks]

#### as document

In [16]:
document_1 = Document(
    page_content=sentences_1,
    metadata={"source": "wikipedia"},
)
document_2 = Document(
    page_content=sentences_2,
    metadata={"source": "tweet"},
)
document_3 = Document(
    page_content=sentences_3,
    metadata={"source": "google"},
)

In [17]:
docs = [document_1, document_2,document_3]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

chunks_of_documents=text_splitter.split_documents(docs)

### setup embedding model

In [10]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en",encode_kwargs={'normalize_embeddings':True})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

### initialize vector store

In [12]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

### Add items to vector store

#### adding text data

In [19]:
text_namespace = "genai_pinecone_text"

In [20]:
vector_store.add_texts(texts=flat_chunks,namespace=text_namespace)

['4728a011-3ba2-416f-bbaa-770be84a43e9',
 '21a31ad5-1aff-46ae-a057-d56d5faefdf4',
 '9b901a24-5ad0-48ac-99a9-affdc7718e70']

#### adding document data

In [21]:
doc_namespace = "genai_pinecone_doc"

In [22]:
vector_store.add_documents(documents=chunks_of_documents,namespace=doc_namespace)

['6c267752-a379-4424-a738-56a68501726c',
 '1bfe79ba-db7c-4f18-92de-7038a3db97cd',
 '1174970e-3c5b-497b-99bc-a4b3008569de']

above output shows `ids` of upserted chunks of documents, we can use these to delete the particular data

### Delete items from vector store

In [24]:
vector_store.delete(ids=['6c267752-a379-4424-a738-56a68501726c'],namespace=doc_namespace)

### Query vector store

#### Query directly

In [25]:
results = vector_store.similarity_search(
    "Something about rajasthan",
    k=2,
    namespace=text_namespace
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* The history of human settlement in Rajasthan goes back more than 100,000 years, with the Indus Valley Civilization marking one of the earliest urban settlements. Sites like Kalibangan show evidence of fire altars, and Rajasthan's strategic location made it a hub for trade. The region's culture and history have been shaped by its rich archaeological past, which continues to be explored. [{}]
* Gujarat, located on the western coast of India, is rich in ancient history. Its settlements date back to the Indus Valley Civilization, with cities like Dholavira showing advanced urban planning. Over 4,000 years ago, it became a center of maritime trade. Gujarat played a crucial role in the spread of culture across the Arabian Sea, with links to Mesopotamia, the Persian Gulf, and even Africa. [{}]


#### Similarity search with score

In [26]:
results = vector_store.similarity_search_with_score(
    "Something about rajasthan", k=1, namespace=text_namespace
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.829067] The history of human settlement in Rajasthan goes back more than 100,000 years, with the Indus Valley Civilization marking one of the earliest urban settlements. Sites like Kalibangan show evidence of fire altars, and Rajasthan's strategic location made it a hub for trade. The region's culture and history have been shaped by its rich archaeological past, which continues to be explored. [{}]


#### Query by turning into retriever

In [27]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings,namespace=text_namespace)
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)
retriever.invoke("Something about rajasthan")

[Document(id='21a31ad5-1aff-46ae-a057-d56d5faefdf4', metadata={}, page_content="The history of human settlement in Rajasthan goes back more than 100,000 years, with the Indus Valley Civilization marking one of the earliest urban settlements. Sites like Kalibangan show evidence of fire altars, and Rajasthan's strategic location made it a hub for trade. The region's culture and history have been shaped by its rich archaeological past, which continues to be explored.")]